In [15]:
import numpy as np
import pandas as pd
import glob

from backend.app import num_random_images

In [16]:
path = './resources/unsplash-dataset/'
documents = ['photos', 'keywords', 'collections', 'conversions', 'colors']
datasets = {}

for doc in documents:
  files = glob.glob(path + doc + ".tsv*")

  subsets = []
  for filename in files:
    df = pd.read_csv(filename, sep='\t', header=0)
    subsets.append(df)

  datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

In [17]:
datasets['photos'].head()

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash
0,bygTaBey1Xk,https://unsplash.com/photos/bygTaBey1Xk,https://images.unsplash.com/uploads/1413387620...,2014-10-15 15:40:40.111061,t,4635,3070,1.51,NaN,jaspervandermeij,...,NaN,NaN,1708356,19085,sea and rock cliff with grasses under cloudy sky,Neist Point,57.428387,-6.783028,30.348906,LcE{wnIVRixt~WR+NGjbxukCWBWB
1,gXSFnk2a9V4,https://unsplash.com/photos/gXSFnk2a9V4,https://images.unsplash.com/reserve/jEs6K0y1Sb...,2014-07-10 18:36:06,t,2448,3264,0.75,Coastline view,kimberlyrichards,...,United States,Tillamook,9895033,74702,aerial photography of seashore,NaN,NaN,NaN,NaN,LXE4G#IARjj]GdWFxaWBDOxaofj[
2,grg6-DNJuaU,https://unsplash.com/photos/grg6-DNJuaU,https://images.unsplash.com/uploads/1412192004...,2014-10-01 19:33:56.393181,t,5184,3456,1.50,NaN,marcusdallcol,...,NaN,NaN,8967968,38338,man surfboarding on ocean wave during daytime,NaN,NaN,NaN,NaN,LcHx?5R%Rjof01bHWBof4ooMoeax
3,sO42hhChB1c,https://unsplash.com/photos/sO42hhChB1c,https://images.unsplash.com/reserve/ijl3tATFRp...,2014-08-19 21:15:40,t,4896,3264,1.50,Hazy Ocean Waters,arturpokusin,...,NaN,NaN,2071752,10860,body of water,NaN,NaN,NaN,NaN,LyOzVsj[aefQ_4j[ayj[IUayj[ay
4,tkk8_HakQ98,https://unsplash.com/photos/tkk8_HakQ98,https://images.unsplash.com/reserve/6vaWXsQuSW...,2014-05-05 18:31:06,t,2000,1333,1.50,Silhouettes In Desert,carlov,...,NaN,NaN,2720281,9081,car on desert during sunset,NaN,NaN,NaN,NaN,"LYEV]I%19ZR+-=s,RkWW00WB%2j["


In [19]:
datasets['keywords'].head()

,photo_id,keyword,ai_service_1_confidence,ai_service_2_confidence,suggested_by_user
0,zzwTUqvzIFg,rock,15.485713,NaN,f
1,zzwTUqvzIFg,cross,19.598213,NaN,f
2,zzwTUqvzIFg,eruption,34.787167,NaN,f
3,zzwTUqvzIFg,sunset,30.080654,NaN,f
4,zzwTUqvzIFg,eclipse,39.832775,NaN,f


In [20]:
datasets['collections']

,photo_id,collection_id,collection_title,photo_collected_at
0,--2IBUMom1I,1230101,Travel,2017-09-27 11:24:17.575047
1,--2IBUMom1I,9832457,business,2020-04-04 14:26:10.506402
2,--2IBUMom1I,2143051,Travel / Places,2018-05-22 23:20:05.898545
3,--2IBUMom1I,FBJEaBSjBvg,Settings,2022-06-04 03:56:40.892078
4,--2IBUMom1I,162470,Majestical Sunsets,2016-03-15 17:04:25.089589
...,...,...,...,...
2524854,zzwTUqvzIFg,10603475,night sky,2020-06-03 21:34:05.585003
2524855,zzwTUqvzIFg,9826446,Expedition,2020-04-03 16:08:51.634711
2524856,zzwTUqvzIFg,s4U-q-TBvEs,8,2021-12-22 06:32:18.223008
2524857,zzwTUqvzIFg,3426848,scenery,2018-10-27 14:10:49.443713


In [21]:
datasets['conversions'].head()

,converted_at,conversion_type,keyword,photo_id,anonymous_user_id,conversion_country
0,2023-05-09 11:03:40.445,download,Mond,jlV2k_Fx0fc,4589085a-75df-417b-93de-22adf2fc627d,DE
1,2023-05-09 11:12:05.109,download,16.9 camel desert,yNGQ830uFB4,e05af0fe-4930-421d-b20d-f904f316e2c3,CN
2,2023-05-09 11:17:33.417,download,bird,BFsm5vldl2I,64fd6739-db67-46e0-99f2-022efb498447,RU
3,2023-05-09 11:32:03.943,download,night sky,-cKXtsJWU-I,2f9c6ac4-02c8-4d0f-82b3-0482a82ab0bf,IN
4,2023-05-09 11:36:56.557,download,zoom background office,CEeoDFpVxxw,a7abbff5-4a50-4c65-b463-18139e2978e9,IN


In [22]:
datasets['colors'].head()

,photo_id,hex,red,green,blue,keyword,ai_coverage,ai_score
0,XDPk8ndzNho,5B534C,91,83,76,darkolivegreen,0.065067,0.030752
1,IfL3QovlAbI,371511,55,21,17,black,0.105533,0.203291
2,GKzgF32piaE,8ACCD5,138,204,213,skyblue,0.044867,0.128655
3,T5WR9adosj8,A59A99,165,154,153,darkgray,0.050800,0.048626
4,T5WR9adosj8,7F7575,127,117,117,gray,0.050533,0.030054


In [29]:
start_index = 5639
end_index = 10000

In [30]:
import os
import requests

# resize all images to max 2000 dimension, pillow thumbnail
from PIL import Image

image_urls = datasets['photos']['photo_image_url'].tolist()
i = start_index
# Watch out, won't be in sync with the index of the image as previously some images failed to download. Will cause url_index to need to be later than i
# Seems to be a problem with url_index 5639 only
url_index = start_index
while i < end_index:
    try:
        response = requests.get(image_urls[url_index], stream=True)
        response.raise_for_status()  # Check for errors

        # Save the image
        with open(f"resources/random_images/image_{i}.jpg", "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)

    except Exception as e:
        print(f"Failed to download {image_urls[i]}: {e}")
        url_index += 1
        continue
    image_path = f'./resources/random_images/image_{i}.jpg'
    image = Image.open(image_path)
    image = image.convert('RGB')
    image.thumbnail((2000, 2000))
    image.save(f'./resources/random_images/image_{i}.jpg')
    i += 1
    url_index += 1


Failed to download https://images.unsplash.com-grass-sun.jpg: HTTPSConnectionPool(host='images.unsplash.com-grass-sun.jpg', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x17dfdfc50>: Failed to resolve 'images.unsplash.com-grass-sun.jpg' ([Errno 8] nodename nor servname provided, or not known)"))


/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (143040000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (91177320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (99996120 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib

In [32]:
from depth_map_generator import depth_map_generator
import cv2
import numpy as np

# Generate depth maps in greyscale jpeg
if not os.path.exists('./resources/random_images_depth_maps_greyscale'):
    os.makedirs('./resources/random_images_depth_maps_greyscale')
for i in range(end_index):
    image_path = f'./resources/random_images/image_{i}.jpg'
    image = cv2.imread(image_path)
    depth_map = depth_map_generator.generate_depth_map(image)
    depth_map_greyscale = (depth_map * 255).astype(np.uint8)
    cv2.imwrite(f'./resources/random_images_depth_maps_greyscale/depth_map_greyscale_{i}.jpg', depth_map_greyscale)



In [11]:
# Generate greyscale depth maps to use jpeg compression

for i in range(num_random_images):
    depth_map_path = f'./resources/random_images_depth_maps/depth_map_{i}.npy'
    depth_map = np.load(depth_map_path)
    depth_map_greyscale = (depth_map * 255).astype(np.uint8)
    cv2.imwrite(f'./resources/random_images_depth_maps_greyscale/depth_map_greyscale_{i}.jpg', depth_map_greyscale)